### PROJECT 1 - Artificial Vision

In [9]:
import cv2
import numpy as np
import os
import matplotlib
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt

# ==============================
# 📌 DEFINE VARIABLES AT THE TOP
# ==============================

# Image Path
IMG_PATH = r"C:\Users\oleru\OneDrive - NTNU\08___4klasse_UPV\04__VA_Artificial_Vision\02__JupyterNotebook\02__Project1\imgs\000002.bmp"

# Morphological Kernel
KERNEL = np.ones((5, 5), np.uint8)

# Transformation Parameters
TRANSLATION_X, TRANSLATION_Y = 0, 0
THRESHOLD_VALUE = 50
RESIZE_DIM = (300, 300)

# Morphological Operation Strength
EROSION_ITERATIONS = 1
DILATION_ITERATIONS = 5

# Median Filter Kernel Size (Must be an odd number)
MEDIAN_KERNEL_SIZE = 5

# Object Labeling Size Range
LABEL_MIN_SIZE = 100
LABEL_MAX_SIZE = 15000

# Extract image name without extension
img_filename = os.path.basename(IMG_PATH)
img_name, img_ext = os.path.splitext(img_filename)
output_filename = f"Final_{img_name}{img_ext}"
output_path = os.path.join(os.path.dirname(IMG_PATH), output_filename)

# Define processing sequence
PROCESSING_ORDER = [
    #"translation",
    "median_filter",
    "grayscale",
    "thresholding",
    #"opening",
    #"erosion",
    "dilation",
    #"closing",
    "edge_detection",
    "labeling",
]


# ==============================
# 📌 DEFINE IMAGE PROCESSING FUNCTIONS
# ==============================

def translate(image):
    h, w = image.shape[:2]
    translation_matrix = np.float32([[1, 0, TRANSLATION_X], [0, 1, TRANSLATION_Y]])
    return cv2.warpAffine(image, translation_matrix, (w, h))


def median_filter(image):
    return cv2.medianBlur(image, MEDIAN_KERNEL_SIZE)


def grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


def thresholding(image):
    if len(image.shape) == 3:
        image = grayscale(image)
    _, binary_img = cv2.threshold(image, THRESHOLD_VALUE, 255, cv2.THRESH_BINARY)
    return binary_img


def morph_opening(image):
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, KERNEL)


def erosion(image):
    return cv2.erode(image, KERNEL, iterations=EROSION_ITERATIONS)


def dilation(image):
    return cv2.dilate(image, KERNEL, iterations=DILATION_ITERATIONS)


def morph_closing(image):
    return cv2.morphologyEx(image, cv2.MORPH_CLOSE, KERNEL)


def edge_detection(image):
    edges = cv2.Canny(image, 100, 200)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    edge_highlighted = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR) if len(image.shape) == 2 else image
    cv2.drawContours(edge_highlighted, contours, -1, (0, 255, 0), 2)
    return edge_highlighted


def label_components(image):
    image_gray = grayscale(image) if len(image.shape) == 3 else image
    inverted_img = cv2.bitwise_not(image_gray)
    _, binary_img = cv2.threshold(inverted_img, 50, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    labeled_img = cv2.cvtColor(binary_img, cv2.COLOR_GRAY2BGR)

    object_count = 0
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        area = w * h
        if LABEL_MIN_SIZE <= area <= LABEL_MAX_SIZE:
            object_count += 1
            cv2.rectangle(labeled_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(labeled_img, f"ID {object_count} ({area}px)", (x, y - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    return labeled_img


def convert_to_bgr(image):
    return cv2.cvtColor(image, cv2.COLOR_GRAY2BGR) if len(image.shape) == 2 else image


def resize_image(image, size=RESIZE_DIM):
    return cv2.resize(image, size)


# ==============================
# 📌 PROCESS IMAGE SEQUENTIALLY
# ==============================

img = cv2.imread(IMG_PATH)
if img is None:
    print("Error: Image could not be loaded. Please check the file path!")
    exit()
else:
    print("✅ Image loaded successfully!")

processed_image = img
step_images = [convert_to_bgr(resize_image(img))]
titles = ["Original Image"]

for step in PROCESSING_ORDER:
    if step == "translation":
        processed_image = translate(processed_image)
    elif step == "median_filter":
        processed_image = median_filter(processed_image)
    elif step == "grayscale":
        processed_image = grayscale(processed_image)
    elif step == "thresholding":
        processed_image = thresholding(processed_image)
    elif step == "opening":
        processed_image = morph_opening(processed_image)
    elif step == "erosion":
        processed_image = erosion(processed_image)
    elif step == "dilation":
        processed_image = dilation(processed_image)
    elif step == "closing":
        processed_image = morph_closing(processed_image)
    elif step == "edge_detection":
        processed_image = edge_detection(processed_image)
    elif step == "labeling":
        processed_image = label_components(processed_image)

    step_images.append(convert_to_bgr(resize_image(processed_image)))
    titles.append(step.replace("_", " ").capitalize())

# ==============================
# 📌 DISPLAY RESULTS IN GRID
# ==============================

fig, axes = plt.subplots(nrows=2, ncols=len(step_images) // 2 + 1, figsize=(15, 8))
axes = axes.flatten()

for i, (img, title) in enumerate(zip(step_images, titles)):
    axes[i].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axes[i].set_title(title)
    axes[i].axis("off")

plt.tight_layout()
plt.show()

# ==============================
# 📌 SAVE FINAL IMAGE
# ==============================

cv2.imwrite(output_path, processed_image)
print(f"✅ Final image saved as: {output_path}")

✅ Image loaded successfully!
✅ Final image saved as: C:\Users\oleru\OneDrive - NTNU\08___4klasse_UPV\04__VA_Artificial_Vision\02__JupyterNotebook\02__Project1\imgs\Final_000002.bmp
